# MLR and Logistic Regression with Python
brian higginbotham

# Introduction

In this notebook, we will practice fitting **MLR** and **Logistic Regression** models using the [wine quality dataset](https://archive.ics.uci.edu/dataset/186/wine+quality) from the Machine Learning Repository. We will first fit four different types of linear regression models to predict the quantitative or continuous response **```alchohol```** and compare their performance. Then we will fit four different logistic models to predict the categorical response **```type```** and compare their performance.

To begin with, let's load in all the libraries we will need.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import mean_squared_error, mean_absolute_error, log_loss, accuracy_score
from sklearn.linear_model import LinearRegression, LassoCV, Lasso, Ridge, RidgeCV, ElasticNet, ElasticNetCV, LogisticRegression, LogisticRegressionCV
from sklearn.preprocessing import PolynomialFeatures

# Data Prep

Our dataset has two separate CSV files - one for red wines and one for white wines. We will read in each file separately, add a categorical variable denoting the wine type, and the concat the two data frames into a single data frame.

In [18]:
red = pd.read_csv("winequality-red.csv", delimiter=';')
red['type'] = 'red'

In [19]:
white = pd.read_csv("winequality-white.csv", delimiter=';')
white['type']='white'

In [20]:
wine_data = pd.concat([red, white], ignore_index=True)
# set ignore_index to True to write a new index for the data frame, otherwise, the original indexing will stay in place causing duplicate
# indexing values from the red and white data sets.

Let's take a quick look at the concatenated data frame to see that everything excecuted correctly.

Columns look good and the size (```RangeIndex```) also looks correct. (We can go back and note the size of each separate data set and see that their sum is equal to the new data set size.)

In [21]:
wine_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         6497 non-null   float64
 1   volatile acidity      6497 non-null   float64
 2   citric acid           6497 non-null   float64
 3   residual sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free sulfur dioxide   6497 non-null   float64
 6   total sulfur dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
 12  type                  6497 non-null   object 
dtypes: float64(11), int64(1), object(1)
memory usage: 660.0+ KB


A quick peak at the data also shows everything to be in expected order, especially note that the added categorical column ```type``` looks correct.

In [22]:
wine_data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,red
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,red
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,red
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,white
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,white
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,white
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,white


We'll convert our categorical variable ```type``` to a binary response: 1 for "red" and 0 for "white." This will allow us to utilize this feature in our linear regression models to predict ```alchohol``` **AND** it will allow us to use this variable as our response in the logistic regression models.

We'll just overwrite the ```type``` column utilizing ```pd.get_dummies()```

In [23]:
wine_data['type'] = pd.get_dummies(data=wine_data['type'], dtype=int)['red']

Another look to make sure the transformation is correct.

In [24]:
wine_data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,0
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,0
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,0
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,0


Next, we'll split the data set up into training and test sets. This will allow us to run **Cross Validation** on the training sets to determine best model parameters while saving the test set for final model evaluation.

For the MLR models, we will be predicting ```alcohol```, so we will drop that column from the predictor sets (```X_``` sets). Conversely, we will set up our prediction output (```y_```) as the column ```alcohol```.

If we inspect the source files, we would notice that there is about three times more white wine data than red wine data. When randomly splitting the data, we want to maintain this proportion in both the test and training sets so as not to skew the model's performance. We can do this using the ```stratify=``` argument.

In [25]:
X_train, X_test, y_train, y_test = train_test_split(
  wine_data.drop("alcohol", axis = 1),
  wine_data["alcohol"],
  test_size=0.20,
  random_state=15,
  stratify=wine_data['type'])

Again, we can take a quick look at the output to see that the data was split appropriately.

In [27]:
# X_train.info()
# X_test.info()
# We won't print the results here in order to save space, but you can quickly check the sizes of each data frame to ensure a 20/80 split.

Lastly, we're going to scale our training and test predictors to 0 with a standard deviation of 1. This will help with the model fitting by reducing the variance of our coefficients.

***Because the model is fit on the scaled training data, we will need to scale our test data using the training data mean and std***   
Therefore, we need to collect the ```mean``` and ```std``` of the testing data to use with the training data ***before*** we scale the training data.

In [29]:
means = X_train.apply(np.mean, axis = 0)
stds = X_train.apply(np.std, axis = 0)

We'll use these ```pandaSeries``` a little further down.

For now, we'll go ahead and scale our training data using a ```lambda``` function.

In [30]:
X_train = X_train.apply(lambda x: (x-np.mean(x))/np.std(x), axis = 0)
X_train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,quality,type
2228,-1.086711,-0.174704,0.084614,-0.884794,-0.562192,-0.415710,-0.285777,-1.020920,-0.242051,0.470217,0.208377,-0.571351
6128,0.071818,-0.478993,1.454725,1.509241,-0.251630,1.672485,0.865182,1.227780,-0.985406,-0.534629,0.208377,-0.571351
2425,-0.314358,-0.539851,0.427142,0.551627,-0.279863,-0.360025,0.698249,0.301844,0.687142,0.403227,0.208377,-0.571351
2939,-0.005417,-1.087572,0.016108,-0.968065,-0.449261,1.087790,0.100804,-0.822506,0.315464,-1.338505,0.208377,-0.571351
5702,-1.163947,-0.174704,-0.257914,-0.281081,-0.731590,-0.471396,-0.514212,-1.344998,0.563249,-1.003557,1.353127,-0.571351
...,...,...,...,...,...,...,...,...,...,...,...,...
2971,2.697819,-0.722425,1.660242,0.572445,-0.336329,0.363883,1.155118,1.095503,-2.162384,1.006135,0.208377,-0.571351
4960,-0.623299,-1.026714,-0.326420,0.926346,-0.251630,-0.248655,-0.250634,0.420893,0.749088,-0.199680,0.208377,-0.571351
278,2.388878,-0.113846,0.906681,0.197726,0.482426,-1.418044,-1.796960,0.963227,0.067680,1.943991,2.497878,1.750237
729,-0.623299,3.202905,-1.970554,-0.468440,0.425960,-0.192970,-1.006225,0.103430,2.421635,-0.266670,0.208377,1.750237


# Linear Regression

## Multiple Linear Regression

Now we are ready to fit some models!

First, let's fit a few ***Multiple Linear Regression*** models with varying features or predictors. We can use ***Cross Validation*** (**CV**) to score the performance of the fitted model. Once we have fitted a few **MLR** models with **CV**, we can then compare the **CV** scores and determine which fit had the best performance. The model with the best performance will then be used to fit the whole test set.

Full MLR

For this model, we use all the features in the training set. The function ```cross_validate()``` takes in the arguments ```LinearRegression()``` as the model type, the predictor values (```X_```), the response value (```y_```), the number of cross validation folds (```cv```), and a scoring metric. For the **MLR** models, we will use **negative mean squared errro.**

In [31]:
full_mlr_cv = cross_validate(
    LinearRegression(),
    X_train,
    y_train,
    cv=5,
    scoring = 'neg_mean_squared_error')

Let's look at the results for each fold.

In [32]:
full_mlr_cv['test_score']

array([-0.25396337, -0.20616874, -0.19578511, -0.47696453, -0.20807482])

Selected MLR

For this model, we'll do the same as above except instead of selecting all the features, we'll only select a few.

In [33]:
part_mlr_cv = cross_validate(
    LinearRegression(),
    X_train[['residual sugar', 'density', 'pH', 'sulphates']],
    y_train,
    cv=5,
    scoring = 'neg_mean_squared_error')

In [34]:
part_mlr_cv['test_score']

array([-0.70380477, -0.71255881, -0.57263128, -0.81646771, -0.5653009 ])

Interaction MLR

In order to inlcude interaction terms, we'll need to transform the predictor variables/features to include interactions (```feature_1 * feature_2```). Use ```PolynomialFeatures()``` to do this: set ```interaction_only``` to ```True``` and apply to ```X_train``` with the selected features. This produces a data frame of the selected variables along with their interactions.

In [35]:
poly = PolynomialFeatures(interaction_only=True, include_bias=False)
interaction_mlr_cv = cross_validate(
    LinearRegression(),
    poly.fit_transform(X_train[['fixed acidity', 'citric acid', 'residual sugar', 'density', 'pH', 'sulphates']]),
    y_train,
    cv=5,
    scoring='neg_mean_squared_error')

In [36]:
interaction_mlr_cv['test_score']

array([-0.34990485, -0.3191202 , -0.27593181, -0.52736628, -0.28915516])

Polynomial MLR

Polynomial features can also be included using ```PolynomialFeatures()```. Instead of the ```interaction_only``` argument, use ```degree=``` and enter the number of degrees you wish to include. Transform the predictor variables the same as above.

In [37]:
poly = PolynomialFeatures(degree=2, include_bias=False)
poly_mlr_cv = cross_validate(
    LinearRegression(),
    poly.fit_transform(X_train[['fixed acidity', 'citric acid', 'residual sugar', 'total sulfur dioxide', 'density', 'sulphates']]),
    y_train,
    cv=5,
    scoring='neg_mean_squared_error')

In [38]:
poly_mlr_cv['test_score']

array([-0.3801997 , -0.34820705, -0.31465346, -0.33930002, -0.35856483])

Determine best MLR model

Now we have scoring results from four different **MLR**'s. Let's transform these results into **Root Mean Squared Errors** (**RMSE**) and select the best score. Take the negative mean of all five negative mean squared errors (thus making it positive) and take it's square root.

In [39]:
print({'full mlr:': np.sqrt(-np.mean(full_mlr_cv['test_score'])),
      'part mlr:': np.sqrt(-np.mean(part_mlr_cv['test_score'])),
      'interaction_mlr_cv:': np.sqrt(-np.mean(interaction_mlr_cv['test_score'])),
      'poly mlr:': np.sqrt(-np.mean(poly_mlr_cv['test_score']))})


{'full mlr:': 0.5178719099853325, 'part mlr:': 0.8210680214674717, 'interaction_mlr_cv:': 0.5935449940117055, 'poly mlr:': 0.5900720413099979}


It looks like the full fit MLR has the best RMSE!

Let's go ahead and fit this model to the full test set.

In [40]:
mlr_best = LinearRegression().fit(
    X_train,
    y_train)

## LASSO

LASSO utilizes a 'penalty' value to determine a best fit model. This tuning parameter can shrink some coefficients to zero, thus eliminating the feature from the model. Thus the LASSO is a variable selection model.

To determine the best parameters for the LASSO, we'll run ```LassoCV``` and give it a range of ```alphas``` to run through and determine optimal fit.

In [41]:
lasso_mod = LassoCV(n_alphas=100, cv=5, random_state=55) \
    .fit(X_train,
         y_train)

Inspect the selected ```alpha``` value.

In [42]:
lasso_mod.alpha_

0.0023165833890366936

Use the selected ```alpha``` value to fit a LASSO regression on the full data set.

In [43]:
lasso_best = Lasso(lasso_mod.alpha_).fit(X_train, y_train)

## Ridge

Ridge is another method that shrinks coefficients to determine best fit. It is useful when there is a high degree of multicollinearity. Similar to running a LASSO, we'll utilize ```RidgeCV()``` and pass it a range of ```alphas``` to run through. ```RidgeCV()``` will select the best parameters.

In [44]:
ridge_mod = RidgeCV(alphas = np.arange(0.1, 10, 0.1).tolist(), cv=5) \
  .fit(X_train, y_train)

Inspect the selected ```alpha``` value.

In [45]:
ridge_mod.alpha_

5.0

Use the selected ```alpha``` value to to fit the Ridge Regression on the full data set.

In [46]:
ridge_best = Ridge(ridge_mod.alpha_).fit(X_train, y_train)

## ElasticNet

Elastic Net is a combination of both LASSO and Ridge ```alpha``` values. The proportion of the ```alpha``` value that is used in calculating the LASSO penalty and the Ridge penalty is called the l1 ratio. (l1 for the LASSO portion and 1-l1 for the Ridge portion).

To cross validate a Ridge model, use ```ElasticNetCV()``` and pass it a list of l1 ratio values. The cross validation will select the best l1 ratio and alpha value.

In [47]:
EN_mod = ElasticNetCV(l1_ratio = [.1, .5, .7, .9, .95, .99, 1], cv =5, random_state = 55) \
  .fit(X_train, y_train)

Inspect the ```alpha``` value as on previous models.

In [48]:
EN_mod.alpha_

0.0023165833890366936

But we can also inspect the selected l1 ratio.

In [49]:
EN_mod.l1_ratio_

1.0

Here, the l1 ratio is 1, which is the equivalent of a LASSO model (i.e., there was no combination of the two models).

Normally, we would go ahead and fit the model on the full data set using the selected ```alpha``` and ```l1_ratio```. But since that would be the same as the LASSO model, we'll alter the ```l1_ratio``` to 0.9 so the model will differ from the LASSO.

In [50]:
EN_best = ElasticNet(alpha=EN_mod.alpha_, l1_ratio=0.9).fit(X_train, y_train)

## Compare Best Linear Models

Before we compare the results of the models, we'll need to scale our test set the same as we did for our train set. Remember earlier that we saved the mean and stds of the train set? Now we'll use those values to scale the test set!

Write a quick function to take in a series or column of data and scale each element by subtracting it from the mean and dividing by the std. Then run that function in a ```for loop```, iterating over the columns of the test set.

In [51]:
def my_std_fun(x, means, stds):
    return(x-means)/stds
#loop through the columns and use the function on each
for x in X_test.columns:
    X_test[x] = my_std_fun(X_test[x], means[x], stds[x])
X_test.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,quality,type
4969,-0.932241,-0.296420,-0.463431,-0.843159,-0.646891,-0.025914,-0.022199,-2.231250,-0.861513,-0.668608,0.208377,-0.571351
2982,-0.082652,0.129585,1.660242,-0.863977,-0.279863,-0.304340,-0.233062,-0.789437,0.129626,-1.338505,0.208377,-0.571351
5553,-0.700535,-0.052989,-0.805959,2.591762,0.284796,0.308197,0.803680,1.459263,-0.489836,-0.132690,-0.936374,-0.571351
5727,-0.468829,-0.052989,0.016108,2.112954,-0.053999,1.756013,1.963425,0.877247,0.191572,0.202258,-0.936374,-0.571351
2077,-0.082652,1.285883,-1.285498,-0.780706,0.482426,-0.527081,0.873968,-0.227262,-1.542922,0.336238,0.208377,-0.571351


Now let's collect the predictions from each model.

In [52]:
mlr_pred = mlr_best.predict(X_test)
lasso_pred = lasso_best.predict(X_test)
ridge_pred = ridge_best.predict(X_test)
EN_pred = EN_best.predict(X_test)

RMSE

First, compare the results using **RMSE**, the same method we used in determing best **MLR** model. Use the ```mean_squared_error``` function from ```sklearn.metrics```. Remember to take the square root of the result!

In [53]:
print({'mlr:': np.sqrt(mean_squared_error(y_test, mlr_pred)),
       'lasso:': np.sqrt(mean_squared_error(y_test, lasso_pred)),
       'ridge:': np.sqrt(mean_squared_error(y_test, ridge_pred)),
      'ENet:': np.sqrt(mean_squared_error(y_test, EN_pred))})

{'mlr:': 0.46070896241429804, 'lasso:': 0.461690327155807, 'ridge:': 0.4612060627638537, 'ENet:': 0.46172844172773714}


The best RMSE was the MLR.  The LASSO and ENet are really close in values. Had we kept the ```l1_ratio``` equal to 1.0, they would have been the same because they would have been the same model!

MAE

Next, compare the results using ```mean_absolute_error()```. This measurement takes the absolute value of the difference between each prediction and actual value, sums them together and takes the average.

In [54]:
print({'mlr:': mean_absolute_error(y_test, mlr_pred),
       'lasso:': mean_absolute_error(y_test, lasso_pred),
       'ridge:': mean_absolute_error(y_test, ridge_pred),
      'ENet:': mean_absolute_error(y_test, EN_pred)})

{'mlr:': 0.3491597963617877, 'lasso:': 0.3505833813356069, 'ridge:': 0.35015495828488635, 'ENet:': 0.3506906955624804}


Again, the results are all pretty similar, but the full fit MLR has the best score.

# Logistic Regression

For Logistic Regression, our target response will be wine type: red or white. We already transformed this variable to a binary category with red = 1 and white = 0.

We'll need to create new Test/Train sets, dropping ```['type']``` from the predictors (```X_```) and adding it to the response (```y_```).

In [55]:
X_train, X_test, y_train, y_test = train_test_split(
  wine_data.drop("type", axis = 1),
  wine_data["type"],
  test_size=0.20,
  random_state=36,
  stratify=wine_data['type'])

Scale the data as before. Keeping a record of the means and stds to use later in ```X_test```.

In [56]:
means = X_train.apply(np.mean, axis = 0)
stds = X_train.apply(np.std, axis = 0)
X_train = X_train.apply(lambda x: (x-np.mean(x))/np.std(x), axis = 0)

## Basic Logistic Regression

Use the same ```cross_validate()``` function as in the linear regression, but instead of using ```LinearRegression()``` as an argument, we'll use ```LogisticRegression(penalty='none')```. It may be easier to create an instance of this function as done below.

We'll score these models on **negative log loss** which is based on the difference between the predicted probability of a result and an the actual result. The smaller the difference, the smaller the score.

Let's fit the first one on all the features/variables.

In [ ]:
log_reg1 = LogisticRegression(penalty='none')
cv1 = cross_validate(log_reg1, X = X_train, y = y_train, cv=5, scoring='neg_log_loss')

Inspect the negative log loss score for each fold.

In [59]:
cv1['test_score']

array([-0.0619739 , -0.04531764, -0.0169939 , -0.03049174, -0.02807723])

Now let's fit the same model on a limited number of features/variables.

In [ ]:
cv2 = cross_validate(log_reg1, X = X_train[['residual sugar', 'density', 'pH']], y=y_train, cv=5, scoring='neg_log_loss')

In [61]:
cv2['test_score']

array([-0.16722455, -0.13379049, -0.14076984, -0.13044344, -0.14346168])

We'll use the same set up with ```PolynomialFeatures()``` as we did before to include interaction terms.

In [ ]:
poly=PolynomialFeatures(interaction_only=True, include_bias=False)
X_transform = poly.fit_transform(X_train[['sulphates', 'citric acid', 'chlorides', 'residual sugar', 'pH', 'density']])
cv3 = cross_validate(log_reg1, X=X_transform, y=y_train, cv=5, scoring='neg_log_loss')

In [63]:
cv3['test_score']

array([-0.09230727, -0.084699  , -0.07265015, -0.08916984, -0.1049967 ])

Again, we'll use the same ```PolynomialFeatures()``` settings to transform the predictors into polynomials, but here we will need to set up a new ```LogisticRegression()``` instance. This one will need to include a ```solver``` and a ```max_iter```.

In [ ]:
poly=PolynomialFeatures(degree=2, include_bias=False)
X_transform = poly.fit_transform(X_train[['residual sugar', 'pH', 'density']])
log_reg2 = LogisticRegression(penalty='none', solver='lbfgs', max_iter=5000)
cv4 = cross_validate(log_reg2, X=X_transform, y=y_train, cv=5, scoring='neg_log_loss')

In [65]:
cv4['test_score']

array([-0.15459723, -0.12936377, -0.12788461, -0.11273159, -0.14285132])

To determine the best Logistic Model, just take the mean of the five log loss scores.

In [66]:
print({'cv1:': round(cv1['test_score'].mean(),4),
       'cv2:': round(cv2['test_score'].mean(),4),
       'cv3:': round(cv3['test_score'].mean(),4),
       'cv4:': round(cv4['test_score'].mean(),4)})

{'cv1:': -0.0366, 'cv2:': -0.1431, 'cv3:': -0.0888, 'cv4:': -0.1335}


```cv1``` has the best ```log_loss``` result.

So now we'll fit the **Logistic Regression** model it's parameters.

In [ ]:
mlr_log = log_reg1.fit(X=X_train, y=y_train)

Inspect the intercept and coefficients of the model.

In [68]:
print(mlr_log.intercept_, mlr_log.coef_)

[-4.34976708] [[-0.23567289  1.20629342 -0.3206709  -4.1965084   0.99136051  1.06339408
  -3.26011156  4.91155915  0.03272167  0.63728033  1.70983294  0.35603188]]


## LASSO

For Regularized Regression, we'll use the ```LogisticRegressionCV()``` function to run cross validations.

To fit a LASSO model, specify ```penalty='l1'```. ```solver=``` can be either 'liblinear' or 'saga' and ```Cs=``` is the range of the regularization parameters the model will iterate through.

In [69]:
l1_cv = LogisticRegressionCV(cv=5, solver='liblinear', penalty='l1', Cs=250, scoring='neg_log_loss', random_state=0) \
  .fit(X_train, y_train)

Here, you can see the best regularization parameter the model selected.

In [70]:
l1_cv.C_[0]

1.3950131878249619

This is the parameter that will be used to fit the LASSO Logistic Regression model on the full train data set, as below.

In [71]:
lasso_log = LogisticRegression(solver='liblinear', penalty='l1', C=l1_cv.C_[0], random_state=33) \
  .fit(X_train, y_train)

## Ridge

Use the same ```LogisticRegressionCV()``` function to fit the Ridge CV model. Specify ```penalty='l2``` for a Ridge model.

In [72]:
l2_cv = LogisticRegressionCV(cv=5, solver='liblinear', penalty='l2', Cs=250, scoring='neg_log_loss', random_state=81) \
  .fit(X_train, y_train)

Inspect the selected best regularization parameter.

In [73]:
l2_cv.C_

array([0.96368643])

Use this parameter to fit the Ridge Logistic Regression model on the full data set.

In [74]:
ridge_log = LogisticRegression(solver='liblinear', penalty='l2', C=l2_cv.C_[0], random_state=77) \
  .fit(X_train, y_train)

## ElasticNet

To run a cross validation for Elastic Net, use the same ```LogisticRegressionCV()``` function, but specify ```solver='saga'```, ```penalty='elasticnet'```, a limit for ```Cs```, and a list of values between 0 and 1 for ```l1_ratios```.

In [ ]:
elasticnet_cv = LogisticRegressionCV(cv=5, solver='saga', penalty='elasticnet', Cs=100, l1_ratios=list(np.arange(0.1, 0.95, 0.2)),
                                     scoring='neg_log_loss', random_state=33) \
                                     .fit(X_train, y_train)

Two 'best' parameters will be selected by the Elastic Net CV, an ```l1_ratio``` along with the ```C_``` value.

In [76]:
print(elasticnet_cv.l1_ratio_)
print(elasticnet_cv.C_)

[0.1]
[0.75646333]


Both will be used to fit an Elastic Net Logistic Regression on the full testing data set.

In [77]:
elasticnet_log = LogisticRegression(solver='saga', penalty='elasticnet', C=elasticnet_cv.C_[0], l1_ratio=elasticnet_cv.l1_ratio_[0],
                                    random_state=90, max_iter=500).fit(X_train, y_train)

## Model Comparison

Now we are ready to compare how each model does in predicting types of wine.

But first, let's scale our test predictors using our earlier calculated training mean and stds.

In [78]:
for x in X_test.columns:
    X_test[x] = my_std_fun(X_test[x], means[x], stds[x])
X_test.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
354,-0.849229,-0.790034,0.558534,-0.848825,0.289747,0.568292,0.866128,-1.168368,0.195564,0.388754,1.186711,0.215330
3820,0.987744,-0.361431,0.149107,1.752419,-0.427376,0.081573,0.795286,1.132596,-0.554369,-0.878720,-0.491885,0.215330
1427,0.298879,0.434547,0.080869,-0.618069,0.633966,-1.407217,-1.648769,0.332261,0.508036,0.322045,0.599202,-0.934794
5763,-0.160364,-0.177743,0.217345,-0.806869,0.203692,-1.006389,0.051443,-0.954945,0.008081,-0.211629,0.179553,-0.934794
2520,0.375420,-0.116514,2.059766,1.081131,-0.570801,1.455840,0.653602,1.032555,-0.554369,-0.611884,-1.331183,0.215330


We'll compare two scoring metrics - Log Loss and Accuracy.

To calculate Log Loss, we'll need to collect the probability predictions (probability of 0 and probability of 1), which we can do using ```.predict_proba()```, for each model.

In [79]:
mlr_proba_preds = mlr_log.predict_proba(X_test)
lasso_proba_preds = lasso_log.predict_proba(X_test)
ridge_proba_preds = ridge_log.predict_proba(X_test)
elasticnet_proba_preds = elasticnet_log.predict_proba(X_test)

Now calculate the Log Loss using the ```log_loss()``` function.

In [80]:
print({'mlr_log:': log_loss(y_test, mlr_proba_preds),
       'lasso_log:': log_loss(y_test, lasso_proba_preds),
       'ridge_log:': log_loss(y_test, ridge_proba_preds),
       'elasticnet_log:': log_loss(y_test, elasticnet_proba_preds)})

{'mlr_log:': 0.04396004207175532, 'lasso_log:': 0.04405722602839568, 'ridge_log:': 0.045252880972079516, 'elasticnet_log:': 0.04633518529089332}


The full logistic regression model has the lowest log loss score.

Now lets look at the accuracy of the four models. This metric counts the number of correct predictions and divides by the total number of predictions.

Collect the predictions using ```.predict()```

In [81]:
mlr_cat_preds = mlr_log.predict(X_test)
lasso_cat_preds = lasso_log.predict(X_test)
ridge_cat_preds = ridge_log.predict(X_test)
elasticnet_cat_preds = elasticnet_log.predict(X_test)

Calculate the accuracy of each model using `accuracy_score()`.

In [82]:
print({'mlr_log:': accuracy_score(y_test, mlr_cat_preds),
       'lasso_log:': accuracy_score(y_test, lasso_cat_preds),
       'ridge_log:': accuracy_score(y_test, ridge_cat_preds),
       'elasticnet_log:': accuracy_score(y_test, elasticnet_cat_preds)})

{'mlr_log:': 0.9923076923076923, 'lasso_log:': 0.9930769230769231, 'ridge_log:': 0.9930769230769231, 'elasticnet_log:': 0.9930769230769231}


The best score is a three-way tie between all of the penalized regression models. This is not that surprising since the results for both the Ridge and Elastic Net CV models indicated that the best **l1_ratio** parameter was 1.0 or close to it and an **l1_ratio** of 1.0 is the same as a LASSO.

In [ ]:
%%shell
jupyter nbconvert --to html ///content/HW7_bhigginbotham.ipynb

[NbConvertApp] Converting notebook ///content/HW7_bhigginbotham.ipynb to html
[NbConvertApp] Writing 758453 bytes to /content/HW7_bhigginbotham.html
